# P419 06 Вместо циклов

Автор: Шабанов Павел Александрович

Email: pa.shabanov@gmail.com

URL: 

+ [Заметки по программированию в науках о Земле](http://progeoru.blogspot.ru/)

+ [GitHub: PyLearn](https://github.com/whitehorn/pyLearn)

+ [О списках с сайта pythonWorld](https://pythonworld.ru/tipy-dannyx-v-python/spiski-list-funkcii-i-metody-spiskov.html)

+ [HABR: Основы Python — кратко. Часть 4. Генераторы списков](https://habr.com/post/30232/) 

+ [HABR: Python: советы, уловки, хаки (часть 1) > 2.1 Генераторы списков](https://habr.com/post/85238/)

+ [HABR: Python: коллекции, часть 4/4: Все о выражениях-генераторах, генераторах списков, множеств и словарей](https://habr.com/post/320288/#7)

Дата последнего обновления: **20.10.2018**

<a id='up'></a>
### План

1. [Вместо списков](#intro)
2. [Генератор списков](#lc)
    + [Создание списков с помощью генератора](#create_list)
    + [Модификация списков с помощью генератора](#modify_list)
    + [Генератор списков с условием](#lc_if)
    + [Вложенные генераторы списков](#lc_doubled)
    + [Выражения-генераторы](#ge) 

3. [Функции, похожие на циклы](#ffunc)
    + [map](#map)
    + [filter](#filter)

### Цель: 

+ научиться пользоваться высокоэффектиным методом обработки и создания списков - генератором списка.

<a id='intro'></a>
## Вместо списков
[Вверх](#up)
Для python вычисления по индексам в нескольких циклах - это неправильный путь. Python будет делать это медленно по сравнению с компилируемыми языками программирования. 

Однако в python есть возможности, чтобы обойти эти слабости. Для стандартной библиотеки (т.е. чистого python, без использования numpy-массивов и других типов данных) такими средствами являются `генераторы списков` и специальные функции `map() и filter()`. 

Ниже приведены результаты времени создания списков длиной 10 млн значений (строковые значения) с помощью трёх методов: циклов, функции map() и генератора списков.

In [23]:
import time

t1 = time.time()
N = 10000000
egg = range(N)
box1 = []
for i in egg:
    box1.append(str(i))
t2 = time.time()
print('Цикл: {:.2f} секунд'.format(t2 - t1))
# Map()
t1 = time.time()
box2 = list(map(lambda x: str(x), egg))
t2 = time.time()
print('Map(): {:.2f} секунд'.format(t2 - t1))

t1 = time.time()
box3 = [str(x) for x in egg]
t2 = time.time()
print('Генератор списка: {:.2f} секунд'.format(t2 - t1))

Цикл: 4.13 секунд
Map(): 3.85 секунд
Генератор списка: 3.12 секунд


Для проверки можно воспользоваться волшебной функцией jupyter notebook - **%%timeit**

In [25]:
N = 10000000
egg = range(N)

In [26]:
%%timeit
box1 = []
for i in egg:
    box1.append(str(i))

3.46 s ± 156 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
box2 = list(map(lambda x: str(x), egg))

4.2 s ± 346 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
box3 = [str(x) for x in egg]

2.9 s ± 69.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Функция `range()` возвращает итератор, а не список. Посмотрим, что будет, если egg станет явным списком.

In [24]:
import time

t1 = time.time()
N = 10000000
egg = list(range(N))
box1 = []
for i in egg:
    box1.append(str(i))
t2 = time.time()
print('Цикл: {:.2f} секунд'.format(t2 - t1))
# Map()
t1 = time.time()
box2 = list(map(lambda x: str(x), egg))
t2 = time.time()
print('Map(): {:.2f} секунд'.format(t2 - t1))

t1 = time.time()
box3 = [str(x) for x in egg]
t2 = time.time()
print('Генератор списка: {:.2f} секунд'.format(t2 - t1))

Цикл: 4.79 секунд
Map(): 4.24 секунд
Генератор списка: 3.71 секунд


<a id='lc'></a>
## Генератор списков
[Вверх](#up)

В python есть целый ряд конструкций, за которые этот язык программирования невозможно не любить. Такие особенности языка, наверное, можно назвать "питоничными" (pythonic).

Одним из них является так называемый `генератор списков` (**list comprehension**).

**Генератор списков** - это заключённая в квадратные скобки (как список) конструкция, которую схематично можно представить так:

> box = [`<операции с элементами последовательности>` **цикл for по элементам последовательности** *условия if-else*]

Это удобный и альтернативный способ создать список из некоторых значений без явного использования цикла.

<a id='create_list'></a>
### Создание списков с помощью генератора
[Вверх](#up)

Помимо создания новых списков, удобно использовать генераторы списков для применения некоторого шаблона-функции к каждому элементу уже созданного списка. Часто в этом случае используются т.н. "анонимные" функции или лямбда-функции (тоже весьма pythonic way на взгляд автора этих строк).

In [ ]:
# Вариант с явным циклом
import random

# Генерация списка с псевдослучайными целыми числами длиной N
N = 50
box = []
for i in range(N):
    rval = random.randint(-50, 100)
    box.append(rval)
print(box[::5])

In [ ]:
# Вариант с генератором списка
import random

# Генерация списка с псевдослучайными целыми числами длиной N
N = 50
box = [random.randint(-50, 100) for i in range(N)]
print(box[::5])

Чтобы создать список из квадратов последовательности от 1 до 37 с шагом 3, нужна всего одна строчка

In [ ]:
res = [x**2 for x in range(1, 37, 3)]
print(res)

<a id='modify_list'></a>
### Модификация списков с помощью генератора
[Вверх](#up)

In [ ]:
print(box)
box = list(range(12))
newBox = ['>>{}<<'.format(elem - 4) for elem in box]
print(newBox)

<a id='lc_if'></a>
### Генератор списков с условием
[Вверх](#up)

В генераторах списков можно использовать условия if-else, они располагаются после описания цикла в теле генератора и отделяются от предыдущих блоков лишь пробелом.

In [ ]:
import random

# Генерация списка fbox только из чётных и отрицательных элементов списка abox
N = 50
abox = [random.randint(-50, 100) for i in range(N)]
fbox = [elem + 451 for elem in abox if ((elem % 2 == 0) and (elem < 0))]
print(fbox[:])

<a id='lc_doubled'></a>
### Вложенные генераторы списков
[Вверх](#up)
    
Как и вложенные циклы, генераторы списков также могут иметь вложенную структуру. Можно сочетать не только циклы, но и условия.

In [1]:
#c = [c * 3 for c in 'list' if c != 'i']
c = [c + d for c in 'list' if c != 'i' for d in 'spam' if d != 'a']
print(c)

['ls', 'lp', 'lm', 'ss', 'sp', 'sm', 'ts', 'tp', 'tm']


<a id='ge'></a>
### Выражения-генераторы
[Вверх](#up)

Помимо генераторов списков  в python существуют `выражения-генераторы (Generator Expressions)`, которые представляют собой особые типы данных, итераторы, которые можно использовать для преобразования к типам данных коллекций: кортежам, множествам и словарям).

**Выражения-генераторы** отличаются по синтаксису от генераторов списков формой используемых скобок. 

Используя круглые скобки () вы получите выражение-генератор. 

Используя фигурные скобки {} можно получить множество или словарь (см. примеры).

Выражения-генераторы не являются "твёрдыми" объектами, они не хранят данные (в отличие от списков, множеств, словарей или кортежей). Генераторы не поддерживают взятие среза, обращение по индексу и т.д. Они умеют лишь выдавать "следующие по списку" значение. Т.е. на каждой итерации генератор выдаёт одно значение, остальные значения (до слеюущей итерации) в генераторах недоступны.

Попробуем использовать фантазию и представить следующее (надеюсь, что это прояснит понимание):

>Пусть списки создаёт гномик, а генераторы - джин.

>Когда мы просим создать список длины N, гномик должен физически создать N листков с номерами от 0 до N-1, на которых содержится какая-то информация.

>В тоже время, когда мы просим создать генератор длины N, то джин готовит не бумажки, а меловую доску. Как мы помним, сам генератор данные не хранит, они появятся, когда это потребуется при итерациях или переборе. Когда будет нужно, джин нарисует на доске цифру ноль, нанесёт на неё информацию, покажет её и будет ждать дальше. Когда будет нужно, он сторёт ноль, нарисует один и нанесёт на доску информацию. Ну и так далее. Т.е. под любое значение N у джина всегда одна доска, а не N листков в случае элементов списка, которые готовит гномик.

>Очевидно, что для больших N, это сильно экономит память. Другое дело, что генераторы можно лишь перебирать, индексацию они не поддерживают (джин умеют рисовать лишь один элемент на доске). Но в каких-то задачах это может серьёзно помочь.`

P.S. Спасибо за объяснение Григорию Петрову и [Python Junior Podcast](https://podcast.python.ru)

In [30]:
# Генерация словаря с помощью генератора
#Пример для словаря из https://habr.com/post/320288/#7

rows = 1, 2, 3, -4, -5
cols = 'a', 'b', 'abc'
# Для наглядности разнесем на несколько строк
my_dict = {
    (col, row): 0  # каждый элемент состоит из ключа-кортежа и нулевого знаечния
    for row in rows if row > 0   # Только положительные значения
    for col in cols if len(col) == 1  # Только односимвольные
    }
print(my_dict)  # {('a', 1): 0, ('b', 2): 0, ('b', 3): 0, ('b', 1): 0, ('a', 3): 0, ('a', 2): 0}

{('a', 1): 0, ('b', 1): 0, ('a', 2): 0, ('b', 2): 0, ('a', 3): 0, ('b', 3): 0}


In [37]:
# Генерация словаря с помощью генератора
#Пример для словаря из https://habr.com/post/320288/#7

list_a = [-2, -1, 0, 1, 2, 3, 4, 5]

gen_a = ((x, x ** 2) for x in list_a)
print(type(gen_a))
dict_a = dict(gen_a)
print(dict_a)
print(type(dict_a))


<class 'generator'>
{-2: 4, -1: 1, 0: 0, 1: 1, 2: 4, 3: 9, 4: 16, 5: 25}
<class 'dict'>


In [39]:
# Генерация множества с помощью генератора
#Пример для словаря из https://habr.com/post/320288/#7

list_a = [-2, -1, 0, 1, 2, 3, 4, 5]
my_set= {i for i in list_a}
print(my_set)   # порядок элементов в множестве неважен, это неупорядоченная коллекция 
print(type(my_set))

{0, 1, 2, 3, 4, 5, -1, -2}
<class 'set'>


In [42]:
# Генерация кортежа с помощью генератора
#Пример для словаря из https://habr.com/post/320288/#7

list_a = [-2, -1, 0, 1, 2, 3, 4, 5]
my_tup = (i for i in list_a)   # это не кортеж, а генератор!
print(my_tup)   # порядок элементов в множестве неважен, это неупорядоченная коллекция 
print(type(my_tup))   # это не кортеж, а генератор!

real_tup = tuple(my_tup)
print(type(real_tup))

<generator object <genexpr> at 0x000002BC07BE0F48>
<class 'generator'>
<class 'tuple'>


<a id='ffunc'></a>
## Функции, похожие на циклы
[Вверх](#up)

В python есть средства функционального программирования (третья общепризнанная парадигма программирования после процедурной и объектно-ориентированной), которые во многом похожи на циклы. Они также позволяют повторять операции и некоторый код, однако делают это не используя циклы. 

Среди таких инструментов стоит рассмотреть следующие функции:

+ **map()**;
+ **filter()**.

Такие функции также являются альтернативой циклам в python, когда необходимо преобразовывать, фильтровать данные и рекурсивно применять какие-то операции.

<a id='map'></a>
### Map()
[Вверх](#up)

**Map()** - это встроенная функция, которая принимает два аргумента:

1. функцию (имя функции без круглых скобок);
2. последовательность, к которой будет применяться функция.

Map() позволяет применить к каждому элементу последовательности некоторую функцию. Таким образом, это функциональный способ замнить цикл. 

Следует помнить, что map() возвращает `тип данных "map"`, а не список или кортеж. Для получения твёрдых данных, необходимо явное преобразование типа с помощью функций list(), tuple() и т.д.

In [43]:
# Преобразовать числа к строковому типу данных

box = range(24)
print(box[0], type(box[0]))

mapped = map(str, box)   # mapped - это не список!!!
print(type(mapped))   # mapped - это особый тип данных "map"

arr = list(mapped)
print(arr[0], type(arr[0]))

0 <class 'int'>
<class 'map'>
0 <class 'str'>


In [ ]:
# Длина списков, которые являются элементами другого списка box
import random

M = 10
N = 6
box = [list(range(N))*random.randint(1, 10) for i in range(M)]

view = list(map(len, box))
print(view)

<a id='filter'></a>
### Filter()
[Вверх](#up)

**Filter()** - это встроенная функция, которая позволяет фильтовать значения. Синтаксис у неё такой же, как у **map()**, она принимает два аргумента:

1. функцию (имя функции без круглых скобок);
2. последовательность, к которой будет применяться функция.

Filter() позволяет фильтровать значения по условию-функции: результат функции проверяется логически и если это Истина, то добавляется в пул.

Следует помнить, что filter(), как и map(), возвращает `тип данных "filter"`, а не список или кортеж. Для получения твёрдых данных, необходимо явное преобразование типа с помощью функций list(), tuple() и т.д.

In [45]:
mixed = ['мак', 'просо', 'мак', 'бобы', 'мак', 'просо', 'мак', 'просо', 'просо', 'просо', 'мак']

# В качестве функции здесь использована не готовая, а созданная на лету анонимная функция
ff = filter(lambda x: x == 'мак', mixed)
print(type(ff))

zolushka = list(ff)
print(zolushka)

<class 'filter'>
['мак', 'мак', 'мак', 'мак', 'мак']


In [ ]:
# Пример фильтрации положительных данных
N = 365
temp = [random.randint(-10, 41) for i in range(N)]
print(temp)

# Если y < 0, то это истина и отбирается фильтром
ff = filter(lambda y: y < 0, temp)

pos = list(ff)
print(pos)

In [ ]:
# Пример фильтрации отсутствующих данных

print(x)
x = list(range(15))
x[2] = None
x[-2] = None

ff = filter(lambda y: y, x)

withoutNone = list(ff)
print(withoutNone)